In [98]:
import os, sklearn, math
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from joblib import dump, load

tf.random.set_seed(42)

data = '211102_Data'
dataname = data+'CMSE'
filename = 'myData/'+data+'.csv'
SD = pd.read_csv(filename)
SD=SD.iloc[:,:28]
SensorData = SD

In [99]:
SD

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx08,Sx09,...,Sx12,Sx13,Sx14,Sx15,Sx16,Sx17,r,theta,theta_enc,z
0,68.91,53.53,52.17,60.41,58.15,56.95,37.80,53.66,57.93,57.05,...,60.84,48.6,77.92,55.36,52.44,71.20,0.9,0,0.00,10
1,72.39,50.49,49.50,58.63,59.09,55.21,35.90,53.66,57.93,57.96,...,60.84,47.6,77.92,53.54,51.42,70.31,0.9,0,0.00,10
2,71.23,49.73,49.50,59.52,60.03,56.95,36.85,52.76,58.84,57.96,...,61.75,48.6,78.78,53.54,53.46,70.31,0.9,0,0.00,10
3,70.07,52.01,50.39,60.41,56.27,54.34,37.80,54.56,58.84,57.05,...,59.93,47.6,77.06,54.45,52.44,69.42,0.9,0,0.00,10
4,68.91,50.49,50.39,61.30,57.21,55.21,37.80,52.76,58.84,58.87,...,61.75,51.6,79.64,54.45,51.42,70.31,0.9,0,0.00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552960,68.91,51.25,49.50,57.74,57.21,56.95,34.95,57.26,61.57,57.05,...,61.75,48.6,82.22,57.18,55.50,70.31,1.8,355,230.58,151
552961,70.07,55.81,53.06,58.63,60.97,53.47,34.00,56.36,62.48,57.96,...,61.75,46.6,79.64,58.09,54.48,66.75,1.8,355,230.58,151
552962,67.75,54.29,51.28,59.52,60.03,56.95,38.75,59.06,59.75,56.14,...,62.66,48.6,78.78,57.18,55.50,67.64,1.8,355,230.58,151
552963,68.91,53.53,51.28,60.41,58.15,54.34,34.95,59.06,60.66,57.05,...,61.75,46.6,79.64,55.36,51.42,68.53,1.8,355,230.58,151


In [100]:
SDext =SD # SensorData[SensorData['z']==0]
mask = SDext["z"].isin(list(range(37,127,3)))
SD = SDext[mask]
SD

,Sx00,Sx01,Sx02,Sx03,Sx04,Sx05,Sx06,Sx07,Sx08,Sx09,...,Sx12,Sx13,Sx14,Sx15,Sx16,Sx17,r,theta,theta_enc,z
103683,67.75,54.29,52.17,60.41,62.85,56.95,38.75,55.46,57.02,57.05,...,63.57,47.6,78.78,49.90,47.34,70.31,0.9,0,130.30,37
103684,70.07,52.77,51.28,61.30,61.91,57.82,32.10,53.66,60.66,57.05,...,63.57,47.6,77.92,52.63,49.38,70.31,0.9,0,130.30,37
103685,71.23,52.01,52.17,59.52,59.09,58.69,37.80,53.66,59.75,57.05,...,63.57,47.6,81.36,51.72,50.40,70.31,0.9,0,130.30,37
103686,67.75,51.25,51.28,62.19,60.03,56.08,35.90,55.46,57.93,57.05,...,63.57,47.6,77.06,48.99,49.38,69.42,0.9,0,130.30,37
103687,68.91,53.53,53.06,63.97,61.91,55.21,37.80,51.86,57.02,57.96,...,65.39,48.6,78.78,48.08,49.38,68.53,0.9,0,130.30,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449280,70.07,53.53,50.39,58.63,47.81,56.95,41.60,57.26,59.75,57.96,...,60.84,47.6,75.34,59.91,52.44,70.31,1.8,355,188.44,124
449281,70.07,55.81,51.28,58.63,48.75,57.82,36.85,58.16,59.75,54.32,...,62.66,47.6,78.78,55.36,53.46,70.31,1.8,355,188.44,124
449282,70.07,55.05,50.39,58.63,50.63,57.82,40.65,57.26,58.84,57.05,...,62.66,48.6,77.06,56.27,52.44,70.31,1.8,355,188.44,124
449283,70.07,54.29,51.28,59.52,50.63,58.69,40.65,56.36,59.75,57.96,...,61.75,46.6,78.78,55.36,51.42,67.64,1.8,355,188.44,124


In [101]:
import numpy as np
toRad = np.pi/180.0
SD.loc[:, 'cos'] = np.cos(SD['theta_enc']*toRad*2)
SD.loc[:, 'sin'] = np.sin(SD['theta_enc']*toRad*2)
SD.loc[:, 'r'] = (SD['r']-SD['r'].mean())/(SD['r'].max()-SD['r'].min())*2
SD.loc[:, 'theta_enc'] = SD['theta_enc'] - 180
SD.loc[:, 'theta_enc'] = (SD['theta_enc']-SD['theta_enc'].mean())/(SD['theta_enc'].max()-SD['theta_enc'].min())*2
SD.loc[:, 'z'] = (SD['z']-SD['z'].mean())/(SD['z'].max()-SD['z'].min())*2

C:\Users\JH_LAB\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\JH_LAB\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [102]:
SDz=(SD.loc[:,'theta_enc'])
SDy = SD.iloc[:,24:30]
SDy

,r,theta,theta_enc,z,cos,sin
103683,-0.999996,0,-0.269045,-1.0,-0.163326,-0.986572
103684,-0.999996,0,-0.269045,-1.0,-0.163326,-0.986572
103685,-0.999996,0,-0.269045,-1.0,-0.163326,-0.986572
103686,-0.999996,0,-0.269045,-1.0,-0.163326,-0.986572
103687,-0.999996,0,-0.269045,-1.0,-0.163326,-0.986572
...,...,...,...,...,...,...
449280,1.000004,355,0.054108,1.0,0.956915,0.290368
449281,1.000004,355,0.054108,1.0,0.956915,0.290368
449282,1.000004,355,0.054108,1.0,0.956915,0.290368
449283,1.000004,355,0.054108,1.0,0.956915,0.290368


In [103]:
X = SD.iloc[:,:24]
y = SD.iloc[:,[26]]
y

,theta_enc
103683,-0.269045
103684,-0.269045
103685,-0.269045
103686,-0.269045
103687,-0.269045
...,...
449280,0.054108
449281,0.054108
449282,0.054108
449283,0.054108


In [104]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size = 0.25, random_state=42)

In [105]:
X_sum = len(X_train)+len(X_valid)+len(X_test)
X_train_per = int(len(X_train)/X_sum*100)
X_valid_per = int(len(X_valid)/X_sum*100)
X_test_per = int(len(X_test)/X_sum*100)

print('X_train: ',len(X_train),'ea',X_train_per,'%   ', 
      'X_valid: ',len(X_valid),'ea',X_valid_per,'%   ',
      'X_test: ',len(X_test),'ea',X_test_per,'%',)

X_train:  207360 ea 59 %    X_valid:  69121 ea 20 %    X_test:  69121 ea 20 %


In [106]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [107]:
import pickle
from joblib import dump, load

file_name = 'myScaler/'+dataname+'_SS.pkl'
dump(scaler, file_name)

['myScaler/211102_DataCMSE_SS.pkl']

In [108]:
import keras.backend as K

def cmse(y_true, y_pred):
    sin_diff = K.sin(y_true - y_pred)
    circular_mean = K.mean(sin_diff, axis=-1)
    circular_diff = K.mean(K.square(tf.math.asin(K.clip(K.sqrt(circular_mean), 0.0, 1.0))), axis=-1)
    return circular_diff


In [109]:
from tensorflow import keras
model = keras.models.Sequential([
    keras.layers.Dense(64, activation="relu", input_shape=X_train.shape[1:],kernel_initializer = "he_normal"),
    keras.layers.Dense(16, activation="relu", input_shape=X_train.shape[1:],kernel_initializer = "he_normal"),
    keras.layers.Dense(16, activation="relu", input_shape=X_train.shape[1:],kernel_initializer = "he_normal"),
    keras.layers.Dense(16, activation="relu", input_shape=X_train.shape[1:],kernel_initializer = "he_normal"),        
    keras.layers.Dense(1)
])
#model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(learning_rate=1e-3, momentum = 0.9,nesterov = True))
model.compile(loss=cmse, optimizer=keras.optimizers.RMSprop(learning_rate=1e-3, rho = 0.9))

In [110]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 64)                1600      
                                                                 
 dense_30 (Dense)            (None, 16)                1040      
                                                                 
 dense_31 (Dense)            (None, 16)                272       
                                                                 
 dense_32 (Dense)            (None, 16)                272       
                                                                 
 dense_33 (Dense)            (None, 1)                 17        
                                                                 
Total params: 3,201
Trainable params: 3,201
Non-trainable params: 0
_________________________________________________________________


## Callback (best, early stopping, tensorboard)

In [111]:
import os 
#root_logdir = os.path.join(os.curdir, 'mylog/'+data)
root_logdir = os.path.join(os.curdir, 'mylog/'+'230406_Data')

def get_run_logdir():
    import time
    run_id= time.strftime("%m%d_%Hh%Mm%Ss"+dataname)
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [112]:
K = keras.backend
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.learning_rate, rate)

In [113]:
dataname

'211102_DataCMSE'

In [114]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
n_epochs = 300
batch_size = 20
onecycle = OneCycleScheduler(math.ceil(len(X_train) / batch_size) * n_epochs, max_rate=0.05)
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid),
                    callbacks=[onecycle,early_stopping_cb,tensorboard_cb])
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[:5]
y_pred = model.predict(X_new)

Epoch 1/300
6480/6480 [==============================] - 7s 1ms/step - loss: 1.9533e-04 - val_loss: 0.0000e+00
Epoch 2/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 4/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 6/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 7/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 8/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 9/300
6480/6480 [==============================] - 7s 1ms/step - loss: 0.0000e+00 - val_loss: 0.0000e+00
E

In [115]:
model.save('D:/ME program/2017-PhD/01_research/02Study/34_ballon_like_cover/DeepLearing/TACTS/LABS_v3_HyperParam/myModel/'+dataname+'.h5')

In [60]:
model = tf.keras.models.load_model('myModel/'+dataname+'.h5', custom_objects={'cmse': cmse})
scaler = load(file_name)
valid_predictions = model.predict(X_valid)
valid_predictions_df=pd.DataFrame(valid_predictions)
#valid_predictions_df.columns = ["valid_theta","valid_z","valid_cos","valid_sin"]
#valid_predictions_df.columns = ["valid_z","valid_cos","valid_sin"]
#valid_predictions_df.columns = ["valid_r","valid_theta","valid_z"]
#valid_predictions_df.columns = ["valid_cos","valid_sin"]
#valid_predictions_df.columns = ["valid_theta","valid_z"]
#valid_predictions_df.columns = ["valid_z"]
valid_predictions_df.columns = ["valid_theta"]
# valid_predictions_df['valid_atan2'] = np.arctan2(-valid_predictions_df['valid_sin'],-valid_predictions_df['valid_cos'])

k= y_valid.to_numpy()
k_df = pd.DataFrame(k)
#k_df.columns = ["theta_enc","z","cos","sin"]
# k_df.columns  = ["z","cos","sin"]
#k_df.columns = ["r","theta_enc","z"]
#k_df.columns  = ["cos","sin"]
#k_df.columns  = ["theta_enc","z"]
#k_df.columns  = ["z"]
k_df.columns  = ["theta_enc"]
# k_df['atan2'] = np.arctan2(-k_df['sin'],-k_df['cos'])

ValidPre = pd.concat( [k_df,valid_predictions_df],axis=1)
ValidPre.head(5)
corr = ValidPre.corr()
print("corr")
for i in range(0,1):    
    print(format(corr[valid_predictions_df.columns[i]][k_df.columns[i]]*100,'.2f')+'%',k_df.columns[i])


2161/2161 [==============================] - 2s 831us/step
corr
-18.14% z
-52.82% cos
-7.27% sin
2.34% atan2


In [61]:
corr

,z,cos,sin,atan2,valid_z,valid_cos,valid_sin,valid_atan2
z,1.000000,-0.000599,-0.004088,0.006216,-0.181440,-0.181465,0.181595,-0.121926
cos,-0.000599,1.000000,0.000259,0.001170,-0.528280,-0.528156,0.527485,-0.182949
sin,-0.004088,0.000259,1.000000,-0.779429,0.072783,0.072777,-0.072658,0.008200
atan2,0.006216,0.001170,-0.779429,1.000000,-0.042629,-0.042623,0.042516,0.023411
valid_z,-0.181440,-0.528280,0.072783,-0.042629,1.000000,1.000000,-0.999997,0.334309
valid_cos,-0.181465,-0.528156,0.072777,-0.042623,1.000000,1.000000,-0.999998,0.334338
valid_sin,0.181595,0.527485,-0.072658,0.042516,-0.999997,-0.999998,1.000000,-0.334696
valid_atan2,-0.121926,-0.182949,0.008200,0.023411,0.334309,0.334338,-0.334696,1.000000
